# Klasteryzacja ofert na podstawie tytułów

W dziejszych czasach nieodłącznym elementem sektora usług konsumenckich jest rekomendacja pewnych przedmiotów. Jedną z firm wykorzystujących tą technikę w świadczonych usługach jest Allegro.pl, największa platforma transakcyjna on-line w Polsce, o czym świadczy dostępna na ich stronie sekcja "polecane dla ciebie". Niestety czasem zdarza się, że system rekomendacyjny poleci użytkownikowi oferty przedstawiające ten sam produkt, ale o innym tytule. Aby uniknąć tego typu sytuacji można próbować łączyć aukcje z tymi samymi przedmiotami w pewne grupy. I właśnie ten projekt miał na celu zmierzenie się z problemem klasteryzacji ofert z portalu Allegro na podstawie ich tytułów. 

# Dane

## Pobranie danych

Dane do projektu należało pobrać przy pomocy Allegro WebAPI, tzn. usługi sieciowej, która umożliwia wymianę informacji między zasobami Allegro a oprogramowaniem zewnętrznym.

W naszym projekcie postanowiliśmy się skupić na książkach o fantastyce, co wynikało głównie ze zwięzłości wybranej kategorii i łatwej formy tytułów ofert. W tym celu pobraliśmy wszystkie możliwe (44357) tytuły dostępnych aukcji w dniu 9 czerwca 2017 roku.

In [14]:
import pandas as pd
from bigramFunctions import getWords, getBigrams
stopwords = pd.read_csv("stopwords.csv").as_matrix()
stopwords.reshape(stopwords.shape[0])
train_data = pd.read_csv("titles_books.csv")
test_data = pd.read_csv("test_data.csv")
train_data = train_data[~train_data["Unnamed: 0"].isin(test_data["Unnamed: 0"])]

print("Liczba ofert: " + str(train_data.shape[0]))
train_data.head(10)

Liczba ofert: 43604


,Unnamed: 0,title
0,24118669,N.Stephenson USTRÓJ ŚWIATA- pomoc rottka.pl
1,24119018,T Canavan NOWICJUSZKA pomoc rottka.pl
2,24119019,T Canavan UCZENNICA MAGA pomoc rottka.pl
3,24119035,T Canavan UCZENNICA MAGA pomoc rottka.pl
4,24174585,ZAMEK ZŁUDZEŃ Mercedes Lackey Josepha Sherman
5,24176443,ZWIADOWCY WCZESNE LATA TURNIEJ W GORLANIE Flan...
6,24186459,*BLOX* Więzień na Marsie. G. Le Rouge. KIC
7,24189517,Brama Ivrel C.J. Cherryh
8,24194754,Silentium Universi - Dariusz Domagalski fantasy
9,24196169,NIEŚMIERTELNY Sharon Sala


## Czyszczenie danych

Zawsze podczas pracy z danymi, w szczególności rzeczywistymi, należy je wyczyścić na tyle na ile jest to możliwe.

Wykorzystana przez nas wstępna obróbka danych to:
1. Usunięcie wszystkich znaków interpunkcyjnych,
2. Zamiana inicjałów autorów na ciąg znaków (np. C.J. -> CJ),
3. Zamiana wielkich liter na małe,
4. Usunięcie pojedynczych znaków i liter,
5. Usunięcie stopwordsów (lista 3717 słów i symboli charakterystycznych dla ofert związanych z książkami),
6. Przedstawienie tytułów ofert jako wektory słów.

# Ogólna idea rozwiązania

Rozwiązanie jest składa się z dwóch etapów: 
1. Budowa wektorów liczbowych reprezentujących słowa (*word2vec*),
2. Klasteryzacja otrzymanych wektorów.  

Wektory budowaliśmy na dwa sposoby: najpierw zastosowaliśmy klasyczne podejście, czyli budowa wektorów dla poszczególnych słów. Tak utworzone zanurzenia nie reprezentowały jednak dobrze relacji między tytułami książek i były bardzo podatne na dodatkowe słowa w tytule. Dlatego przetestowaliśmy również wektory zbudowane dla bigramów, czyli par słów występujących obok siebie.

In [9]:
print(getWords("Tolkien - Władca pierścieni - Trylogia", stopwords))

['tolkien', 'władca', 'pierścieni', 'trylogia']


In [10]:
print(getBigrams("Tolkien - Władca pierścieni - Trylogia", stopwords))

['tolkien władca', 'pierścieni władca', 'pierścieni trylogia']


Bigramy są sortowane alfabetycznie - z punktu widzenia algorytmu nie ma to żadnego wpływu na wyniki, a w ten sposób możemy się uodpornić na różne konwencje zapisywania imienia i nazwiska.

#### Word2Vec

Jest to algorytm pozwalający na przedstawienie słów pochodzących z pewnego korpusu dokumentów jako zbiór wektorów w *n*-wymiarowej przestrzeni. Wykorzystaliśmy implementację dostępną w pakiecie `gensim` opartą o model **CBOW** (*Continuous Bag Of Words*). Proces uczenia przebiega następująco - za pomocą sieci neuronowej próbujemy przewidzieć słowo na podstawie jego najbliższego otoczenia. Mając tak wytrenowany model możemy otrzymać interesujące nas wektory na podstawie wartości warstwy ukrytej.

Głównym celem było uzyskanie takich wektorów, które będą cechowały się niską odległością między wektorami reprezentującymi różne oferty tych samych książek. Reprezentację wektorową tytułu oferty otrzymujemy uśredniając wektory pojedynczych słów tworzących ten tytuł.

#### MeanShift

Jest to algorytm klasteryzacji oparty o estymację jądrową. W każdym punkcie zbioru nakładamy pewne jądro (przykładowo gaussowskie), a następnie sumujemy wartości otrzymane w danych punktach. W ten sposób otrzymujemy powierzchnię reprezentującą nasze dane. Następnym krokiem jest iteracyjne "wciąganie" wszystkich punktów na szczyt otrzymanej powierzchni. W ten sposób otrzymujemy klastry oraz punkty które do nich przynależą. Głównym parametrem algorytmu jest dokładność odwzorowania otrzymanej powierzchni - im większa, tym więcej klastrów otrzymujemy.

<div
style="float:center;margin-right:5px;">
    <div style="float:left;margin-right:5px;">
        <img src="img/ms_2d_bw_2.gif" width="400" height="300" />
    </div>
    <div style="float:left;margin-right:5px;">
        <img src="img/ms_2d_bw_.8.gif" width="400" height="300" />
    </div>
</div>



# Miary jakości

W przypadku gdy znane są prawdziwe klasy jest możliwe zdefiniowanie intuicyjnych miar oceny klasteryzacji:
1. homogeniczność - miara określająca czy każdy klaster zawiera tylko elementy z tej samej jednej klasy,
2. kompletność - miara określająca czy wszyscy członkowie jednej klasy należą do tego samego klastra,
3. V-miara - średnia harmoniczna powyższych miar.

Przedziałem wartości tych miar jest $[0,1]$, gdzie $0$ oznacza złe dopasowanie, a $1$ perfekcyjne. Główną zaletą tych miar jest ich intuicyjność oraz brak wymogów co do założeń struktury modelu do klasteryzacji.

Więcej o tych miarach i wzorach je reprezentujących można przeczytać [tutaj](http://scikit-learn.org/stable/modules/clustering.html#homogeneity-completeness-and-v-measure).


# Wyniki

Procedura trenowania i testowania modelu klastrującego:
1. Na zbiorze treningowym trenujemy algorytmy word2vec, a następnie MeanShift,
2. Mając gotowe wektory przekształcamy testowe tytuły ofert na wektory, a następnie przydzielamy je do otrzymanych w poprzednim kroku klastrów.
3. Dodatkowo obliczamy również wybrane przez nas miary na pewnym małym podzbiorze zbioru testowego w celu sprawdzenia działania klasteryzacji na zbiorze symulującym wyniki rekomendacji.



|                | H    | C    | V    |
|----------------|------|------|------|
| Pełny zbiór testowy   | 0.63 | 0.78 | 0.69 |
| Pozdbiór | 0.76  | 0.58  | 0.65 |

<p style="text-align: center;"> Wyniki przy użyciu wektorów dla słów </p>

|                | H    | C    | V    |
|----------------|------|------|------|
| Pełny zbiór testowy   | 0.63 | 0.78 | 0.69 |
| Pozdbiór | 0.76  | 0.58  | 0.65 |

<p style="text-align: center;"> Wyniki przy użyciu wektorów dla bigramów </p>

Widzimy, że na pełnym zbiorze testowym mamy sytuację, w której otrzymane klastry zawierają po kilka książek, ale poszczególne tytuły należą raczej do jednego klastra. Wskazuje na to odpowiednio duża wartość kompletności oraz trochę mniejsza homogeniczność. Ogólnie jednak otrzymane wyniki są dobre i v-score wynosi 0.69.

Dla mniejszego podzbioru sytuacja się odwraca - mamy klastry które zawierają faktycznie te same książki, ale książki mogą być rozbite na kilka klastrów. W praktyce oznaczałoby to, że po redukcji wyników za pomocą naszego algorytmu nie pozbylibyśmy się w pełni duplikatów, ale jednocześnie nie stracilibyśmy zbyt dużo otrzymanych unikalnych wyników rekomendacji.

# Wizualizacje word2vec

Ponieważ cieżko o dobrą miarę która pokaże jakość samej transformacji słów na wektory, postanowiliśmy odwołać się do wizualizacji.

Pierwszy wykres pokazuje jak wyglądają odległości między wektorami dla ofert dotyczących wybranych książek. W szczególności chcieliśmy sprawdzić, czy da się odróżnić od siebie dzieła różnych autorów, oraz czy da się podzielić książki w obrębie autora.

![](img\heatmapa.png)

Pierwsze, co rzuca się w oczy to wyraźnie wydrębnione grupy książek różnych autorów. Jeśli chodzi o różnice między konkretnymi książkami, to dla książek o Harrym Potterze widzimy dosć wyraźny podział, ale już wśród innych autorów nie jest to aż tak widoczne.  

Kolejny wykres przedstawia rzutowanie 32-wymiarowej przestrzeni tytułów ofert ze zbioru testowego na dwu wymiarową płaszczyznę. Rzutowanie zostało przeprowadzone za pomocą algorytmu `t-sne`, który służy do redukcji wymiarów przy jednoczesnym zachowaniu kształtu i odległości między punktami.

Widzimy wyraźnie wydrębnione grupy książek od róznych autorów, niestety struktura wewnątrz nich jest na ogół dość chaotyczna.

[Interaktywny wykres](bokeh_vis\test_data_bokeh_vis.html)

Ostatni wykres jest podobny w założeniach, ale różni się tym, że przedstawione tutaj są reprezentacje wektorowe oparte o bigramy.
